####  Datos de shapes descargados de: 
https://www.inegi.org.mx/app/mapas/
http://internet.contenidos.inegi.org.mx/contenidos/Productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/CGURA_Junio2016/Nuevo_Leon/702825218867_s.zip

## Importar data frames

In [1]:
import pandas as pd
from functools import partial
from pyproj import Proj, transform

In [2]:
df = pd.read_csv("data/crime.csv")

In [3]:
df.head()

,year,date,lat,lon,news,cassualties,url,neighborhood
0,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES
1,2015,2015-10-02,25.632075,-100.286088,Ejecutan a uno en plaza en Garza Sada,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,REVOLUCION PROLETARIA
2,2015,2015-10-01,25.675784,-100.475788,Lo ejecutan frente a su casa,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,HDA SANTA CATARINA (FOMERREY 29)
3,2015,2015-09-24,25.657586,-100.322029,Lo esperan para ejecutarlo,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,INDEPENDENCIA
4,2015,2015-09-24,25.683207,-100.305650,Matan a 2 clientes en ataque 13 a bar,2.0,http://www.elnorte.com/aplicaciones/articulo/d...,CENTRO DE MONTERREY


In [4]:
print(df.shape)

(2114, 8)


In [5]:
## http://epsg.io/6372
outProj = Proj(init='epsg:6372')
inProj = Proj(init='epsg:4326')

In [6]:
def getCoordinates(row):
    ## Function that gets a coordinate in epsg:4326 and returns a point in epsg:6372
    x, y = transform(inProj, outProj, row["lon"], row["lat"])
    return [x,y]

In [7]:
df_min = df.iloc[:5,].copy()

In [8]:
df_min["point"] = df_min.apply(getCoordinates, axis = 1)

In [9]:
df_min["x"] = [p[0] for p in df_min["point"]]

In [10]:
df_min

,year,date,lat,lon,news,cassualties,url,neighborhood,point,x
0,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES,"[2673990.068792671, 1508739.2403134028]",2.673990e+06
1,2015,2015-10-02,25.632075,-100.286088,Ejecutan a uno en plaza en Garza Sada,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,REVOLUCION PROLETARIA,"[2671295.4132258967, 1510294.803596502]",2.671295e+06
2,2015,2015-10-01,25.675784,-100.475788,Lo ejecutan frente a su casa,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,HDA SANTA CATARINA (FOMERREY 29),"[2652285.5673206407, 1514899.6189773784]",2.652286e+06
3,2015,2015-09-24,25.657586,-100.322029,Lo esperan para ejecutarlo,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,INDEPENDENCIA,"[2667670.5901743174, 1513064.839283476]",2.667671e+06
4,2015,2015-09-24,25.683207,-100.305650,Matan a 2 clientes en ataque 13 a bar,2.0,http://www.elnorte.com/aplicaciones/articulo/d...,CENTRO DE MONTERREY,"[2669273.8077375377, 1515908.7290589423]",2.669274e+06


#### Pasar de coordenadas a puntos x, y para mapearlos con los paths del INEGI. 
#### Lambert conformal conic projection

In [11]:
df["point"] = df.apply(getCoordinates, axis=1)

In [12]:
df.head()

,year,date,lat,lon,news,cassualties,url,neighborhood,point
0,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES,"[2673990.068792671, 1508739.2403134028]"
1,2015,2015-10-02,25.632075,-100.286088,Ejecutan a uno en plaza en Garza Sada,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,REVOLUCION PROLETARIA,"[2671295.4132258967, 1510294.803596502]"
2,2015,2015-10-01,25.675784,-100.475788,Lo ejecutan frente a su casa,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,HDA SANTA CATARINA (FOMERREY 29),"[2652285.5673206407, 1514899.6189773784]"
3,2015,2015-09-24,25.657586,-100.322029,Lo esperan para ejecutarlo,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,INDEPENDENCIA,"[2667670.5901743174, 1513064.839283476]"
4,2015,2015-09-24,25.683207,-100.305650,Matan a 2 clientes en ataque 13 a bar,2.0,http://www.elnorte.com/aplicaciones/articulo/d...,CENTRO DE MONTERREY,"[2669273.8077375377, 1515908.7290589423]"


## Mapear a shapes de INEGI

In [13]:
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
import shapefile as shp

#### Los archivos que terminan en a se refieren a: Área geoestadística básica

In [14]:
sf = shp.Reader("inegi_NL_2016/conjunto_de_datos/19a", encoding="latin1")

In [15]:
print(sf)

shapefile Reader
    2567 shapes (type 'POLYGON')
    2567 records (5 fields)


#### Son 2567 AGEBs

In [16]:
s = sf.shape(0)
['%.3f' % coord for coord in s.bbox]

['2747500.832', '1425281.068', '2747635.517', '1425592.061']

In [17]:
sf.fields

[('DeletionFlag', 'C', 1, 0),
 ['CVE_ENT', 'C', 2, 0],
 ['CVE_MUN', 'C', 3, 0],
 ['CVE_LOC', 'C', 4, 0],
 ['CVE_AGEB', 'C', 4, 0]]

In [18]:
sf.record(0)

Record #0: ['19', '033', '0001', '131A']

In [19]:
paths = [Path(sf.shape(i).points) for i in range(len(sf.shapes())) ]

In [20]:
paths[0].contains_point((2690417.3087897752, 1512217.7368954867))

False

In [21]:
def point_in_path(row):
    for i in range(len(paths)):
        if(paths[i].contains_point(row["point"])):
            return i

####  Test con df_min

In [22]:
df_min["r"] = df_min.apply(point_in_path, axis = 1)

In [25]:
df_min.head()

,year,date,lat,lon,news,cassualties,url,neighborhood,point,x,r,municipio,localidad,ageb
0,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES,"[2673990.068792671, 1508739.2403134028]",2.673990e+06,645,039,0001,3729
1,2015,2015-10-02,25.632075,-100.286088,Ejecutan a uno en plaza en Garza Sada,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,REVOLUCION PROLETARIA,"[2671295.4132258967, 1510294.803596502]",2.671295e+06,1186,039,0001,4727
2,2015,2015-10-01,25.675784,-100.475788,Lo ejecutan frente a su casa,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,HDA SANTA CATARINA (FOMERREY 29),"[2652285.5673206407, 1514899.6189773784]",2.652286e+06,2318,048,0001,0545
3,2015,2015-09-24,25.657586,-100.322029,Lo esperan para ejecutarlo,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,INDEPENDENCIA,"[2667670.5901743174, 1513064.839283476]",2.667671e+06,2095,039,0001,1968
4,2015,2015-09-24,25.683207,-100.305650,Matan a 2 clientes en ataque 13 a bar,2.0,http://www.elnorte.com/aplicaciones/articulo/d...,CENTRO DE MONTERREY,"[2669273.8077375377, 1515908.7290589423]",2.669274e+06,2083,039,0001,1281


In [65]:
def getRecord(r, j):
    if not np.isnan(r):
        if j != 3:
            return int(sf.record(int(r))[j])
        else:
            return sf.record(int(r))[j]
    else:
        return np.nan

In [62]:
df_min["municipio"] = df_min["r"].apply(getRecord, j = 1)
df_min["localidad"] = df_min["r"].apply(getRecord, j = 2)
df_min["ageb"] = df_min["r"].apply(getRecord, j = 3)

In [63]:
df_min

,year,date,lat,lon,news,cassualties,url,neighborhood,point,x,r,municipio,localidad,ageb
0,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES,"[2673990.068792671, 1508739.2403134028]",2.673990e+06,645,39,1,3729
1,2015,2015-10-02,25.632075,-100.286088,Ejecutan a uno en plaza en Garza Sada,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,REVOLUCION PROLETARIA,"[2671295.4132258967, 1510294.803596502]",2.671295e+06,1186,39,1,4727
2,2015,2015-10-01,25.675784,-100.475788,Lo ejecutan frente a su casa,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,HDA SANTA CATARINA (FOMERREY 29),"[2652285.5673206407, 1514899.6189773784]",2.652286e+06,2318,48,1,545
3,2015,2015-09-24,25.657586,-100.322029,Lo esperan para ejecutarlo,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,INDEPENDENCIA,"[2667670.5901743174, 1513064.839283476]",2.667671e+06,2095,39,1,1968
4,2015,2015-09-24,25.683207,-100.305650,Matan a 2 clientes en ataque 13 a bar,2.0,http://www.elnorte.com/aplicaciones/articulo/d...,CENTRO DE MONTERREY,"[2669273.8077375377, 1515908.7290589423]",2.669274e+06,2083,39,1,1281


#### Ahora sí con df

In [26]:
df["r"] = df.apply(point_in_path, axis = 1)

In [27]:
df.head()

,year,date,lat,lon,news,cassualties,url,neighborhood,point,r
0,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES,"[2673990.068792671, 1508739.2403134028]",645.0
1,2015,2015-10-02,25.632075,-100.286088,Ejecutan a uno en plaza en Garza Sada,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,REVOLUCION PROLETARIA,"[2671295.4132258967, 1510294.803596502]",1186.0
2,2015,2015-10-01,25.675784,-100.475788,Lo ejecutan frente a su casa,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,HDA SANTA CATARINA (FOMERREY 29),"[2652285.5673206407, 1514899.6189773784]",2318.0
3,2015,2015-09-24,25.657586,-100.322029,Lo esperan para ejecutarlo,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,INDEPENDENCIA,"[2667670.5901743174, 1513064.839283476]",2095.0
4,2015,2015-09-24,25.683207,-100.305650,Matan a 2 clientes en ataque 13 a bar,2.0,http://www.elnorte.com/aplicaciones/articulo/d...,CENTRO DE MONTERREY,"[2669273.8077375377, 1515908.7290589423]",2083.0


In [34]:
df[df["neighborhood"].isnull()].shape

(254, 10)

In [35]:
df[df["r"].isnull()].shape

(197, 10)

In [36]:
import numpy as np

In [66]:
df["municipio"] = df["r"].apply(getRecord, j = 1)
df["localidad"] = df["r"].apply(getRecord, j = 2)
df["ageb"] = df["r"].apply(getRecord, j = 3)

In [67]:
df.head(50)

,year,date,lat,lon,news,cassualties,url,neighborhood,point,r,municipio,localidad,ageb
0,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES,"[2673990.068792671, 1508739.2403134028]",645.0,39.0,1.0,3729
1,2015,2015-10-02,25.632075,-100.286088,Ejecutan a uno en plaza en Garza Sada,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,REVOLUCION PROLETARIA,"[2671295.4132258967, 1510294.803596502]",1186.0,39.0,1.0,4727
2,2015,2015-10-01,25.675784,-100.475788,Lo ejecutan frente a su casa,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,HDA SANTA CATARINA (FOMERREY 29),"[2652285.5673206407, 1514899.6189773784]",2318.0,48.0,1.0,0545
3,2015,2015-09-24,25.657586,-100.322029,Lo esperan para ejecutarlo,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,INDEPENDENCIA,"[2667670.5901743174, 1513064.839283476]",2095.0,39.0,1.0,1968
4,2015,2015-09-24,25.683207,-100.305650,Matan a 2 clientes en ataque 13 a bar,2.0,http://www.elnorte.com/aplicaciones/articulo/d...,CENTRO DE MONTERREY,"[2669273.8077375377, 1515908.7290589423]",2083.0,39.0,1.0,1281
5,2015,2015-09-23,25.777561,-100.410404,Asesinan a ex reo de tres balazos,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,LA ALIANZA,"[2658693.510923911, 1526191.7772835381]",2239.0,39.0,1.0,5015
6,2015,2015-09-18,25.702387,-100.330845,Ejecutan a uno en la Col. Garza Nieto,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,PEDRO LOZANO,"[2666732.2379076034, 1517993.7998131234]",2075.0,39.0,1.0,0921
7,2015,2015-09-05,25.584773,-100.235480,Ejecutan a joven tras persecuci&#243;n,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,NOGALES DE LA SIERRA,"[2676417.3226552834, 1505141.5785776984]",2137.0,39.0,1.0,2970
8,2015,2015-09-05,25.691347,-100.280345,Lo acribillan desde un veh&#237;culo,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,NUEVA MADERO,"[2671791.048815214, 1516836.2648395365]",1199.0,39.0,1.0,1347
9,2015,2015-09-07,25.801002,-100.053654,Torturan y ejecutan a dos en Pesquer&#237;a,2.0,http://www.elnorte.com/aplicaciones/articulo/d...,NaN,"[2694271.730618012, 1529215.312199961]",NaN,NaN,NaN,NaN


### No están todos los shapes :(
### pero no faltan tantos :)

In [68]:
ageeml = pd.read_csv("data/AGEEML_NL.csv")

In [69]:
ageeml.head()

,Mapa,Cve_Ent,Nom_Ent,Nom_Abr,Cve_Mun,Nom_Mun,Cve_Loc,Nom_Loc,Ámbito,Latitud,Longitud,Lat_Decimal,Lon_Decimal,Altitud,Cve_Carta,Pob_Total,Pob_Masculina,Pob_Femenina,Total De Viviendas Habitadas
0,190010001,19,Nuevo León,NL,1,Abasolo,1,Abasolo,U,"25°56´43.215N""","100°23´59.958W""",25.945338,-100.399988,502,G14C15,1976,1005,971,529
1,190010008,19,Nuevo León,NL,1,Abasolo,8,La Gloria,R,"25°56´14.103N""","100°22´48.748W""",25.937251,-100.380208,499,G14C15,7,0,0,1
2,190010009,19,Nuevo León,NL,1,Abasolo,9,Los Ligeros,R,"25°55´57.079N""","100°23´39.742W""",25.932522,-100.394373,504,G14C15,3,0,0,1
3,190010011,19,Nuevo León,NL,1,Abasolo,11,La Muralla,R,"25°56´18.560N""","100°23´52.699W""",25.938489,-100.397972,500,G14C15,4,0,0,2
4,190010012,19,Nuevo León,NL,1,Abasolo,12,Seis de Enero (Las Bugambilias),R,"25°56´59.312N""","100°24´26.446W""",25.949809,-100.407346,509,G14C15,0,0,0,0


In [70]:
ageeml.dtypes

Mapa                              int64
Cve_Ent                           int64
Nom_Ent                          object
Nom_Abr                          object
Cve_Mun                           int64
Nom_Mun                          object
Cve_Loc                           int64
Nom_Loc                          object
Ámbito                           object
Latitud                          object
Longitud                         object
Lat_Decimal                     float64
Lon_Decimal                     float64
Altitud                           int64
Cve_Carta                        object
Pob_Total                         int64
Pob_Masculina                     int64
Pob_Femenina                      int64
Total De Viviendas Habitadas      int64
dtype: object

In [71]:
ageeml["Cve_Mun"] = ageeml["Cve_Mun"].astype(float)

In [81]:
merged_min = df_min.merge(ageeml[["Cve_Mun", "Nom_Mun"]], right_on="Cve_Mun", left_on='municipio', how="left")

In [77]:
ageeml["Cve_Loc"] = ageeml["Cve_Loc"].astype(float)

In [83]:
merged_min = merged_min.merge(ageeml[["Cve_Loc", "Nom_Loc"]], right_on="Cve_Loc", left_on='localidad', how="left")

In [84]:
merged_min

,year,date,lat,lon,news,cassualties,url,neighborhood,point,x,r,municipio,localidad,ageb,Cve_Mun,Nom_Mun,Cve_Loc,Nom_Loc
0,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES,"[2673990.068792671, 1508739.2403134028]",2.673990e+06,645,39,1,3729,39.0,Monterrey,1.0,Abasolo
1,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES,"[2673990.068792671, 1508739.2403134028]",2.673990e+06,645,39,1,3729,39.0,Monterrey,1.0,Agualeguas
2,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES,"[2673990.068792671, 1508739.2403134028]",2.673990e+06,645,39,1,3729,39.0,Monterrey,1.0,Los Aldamas
3,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES,"[2673990.068792671, 1508739.2403134028]",2.673990e+06,645,39,1,3729,39.0,Monterrey,1.0,Ciudad de Allende
4,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES,"[2673990.068792671, 1508739.2403134028]",2.673990e+06,645,39,1,3729,39.0,Monterrey,1.0,Anáhuac
5,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES,"[2673990.068792671, 1508739.2403134028]",2.673990e+06,645,39,1,3729,39.0,Monterrey,1.0,Ciudad Apodaca
6,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES,"[2673990.068792671, 1508739.2403134028]",2.673990e+06,645,39,1,3729,39.0,Monterrey,1.0,Aramberri
7,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES,"[2673990.068792671, 1508739.2403134028]",2.673990e+06,645,39,1,3729,39.0,Monterrey,1.0,Bustamante
8,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES,"[2673990.068792671, 1508739.2403134028]",2.673990e+06,645,39,1,3729,39.0,Monterrey,1.0,Cadereyta Jiménez
9,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES,"[2673990.068792671, 1508739.2403134028]",2.673990e+06,645,39,1,3729,39.0,Monterrey,1.0,Carmen


In [91]:
merged = df.merge(ageeml[["Cve_Mun", "Nom_Mun", "Cve_Loc", "Nom_Loc"]], 
                  right_on=["Cve_Mun", "Cve_Loc"], left_on=["municipio", "localidad"], how="left")

In [92]:
print(merged.shape)

(2114, 17)


In [106]:
merged.head(20)

,year,date,lat,lon,news,cassualties,url,neighborhood,point,clave_mun,nom_mun,clave_loc,nom_loc,ageb
0,2015,2015-10-12,25.617672,-100.259318,Persiguen y ejecutan a hombre,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,EJIDAL LOS REMATES,"[2673990.068792671, 1508739.2403134028]",39,Monterrey,1.0,Monterrey,3729
1,2015,2015-10-02,25.632075,-100.286088,Ejecutan a uno en plaza en Garza Sada,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,REVOLUCION PROLETARIA,"[2671295.4132258967, 1510294.803596502]",39,Monterrey,1.0,Monterrey,4727
2,2015,2015-10-01,25.675784,-100.475788,Lo ejecutan frente a su casa,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,HDA SANTA CATARINA (FOMERREY 29),"[2652285.5673206407, 1514899.6189773784]",48,Santa Catarina,1.0,Ciudad Santa Catarina,0545
3,2015,2015-09-24,25.657586,-100.322029,Lo esperan para ejecutarlo,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,INDEPENDENCIA,"[2667670.5901743174, 1513064.839283476]",39,Monterrey,1.0,Monterrey,1968
4,2015,2015-09-24,25.683207,-100.305650,Matan a 2 clientes en ataque 13 a bar,2.0,http://www.elnorte.com/aplicaciones/articulo/d...,CENTRO DE MONTERREY,"[2669273.8077375377, 1515908.7290589423]",39,Monterrey,1.0,Monterrey,1281
5,2015,2015-09-23,25.777561,-100.410404,Asesinan a ex reo de tres balazos,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,LA ALIANZA,"[2658693.510923911, 1526191.7772835381]",39,Monterrey,1.0,Monterrey,5015
6,2015,2015-09-18,25.702387,-100.330845,Ejecutan a uno en la Col. Garza Nieto,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,PEDRO LOZANO,"[2666732.2379076034, 1517993.7998131234]",39,Monterrey,1.0,Monterrey,0921
7,2015,2015-09-05,25.584773,-100.235480,Ejecutan a joven tras persecuci&#243;n,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,NOGALES DE LA SIERRA,"[2676417.3226552834, 1505141.5785776984]",39,Monterrey,1.0,Monterrey,2970
8,2015,2015-09-05,25.691347,-100.280345,Lo acribillan desde un veh&#237;culo,1.0,http://www.elnorte.com/aplicaciones/articulo/d...,NUEVA MADERO,"[2671791.048815214, 1516836.2648395365]",39,Monterrey,1.0,Monterrey,1347
9,2015,2015-09-07,25.801002,-100.053654,Torturan y ejecutan a dos en Pesquer&#237;a,2.0,http://www.elnorte.com/aplicaciones/articulo/d...,NaN,"[2694271.730618012, 1529215.312199961]",,NaN,NaN,NaN,NaN


In [95]:
merged = merged.drop(["r", "municipio", "localidad"], axis=1)

In [101]:
merged.columns

Index(['year', 'date', 'lat', 'lon', 'news', 'cassualties', 'url',
       'neighborhood', 'point', 'clave_mun', 'nom_mun', 'clave_loc', 'nom_loc',
       'ageb'],
      dtype='object')

In [98]:
merged.columns = ['year', 'date', 'lat', 'lon', 'news', 'cassualties', 'url',
       'neighborhood', 'point', 'ageb', 'clave_mun', 'nom_mun', 'clave_loc',
       'nom_loc']

In [100]:
merged = merged[['year', 'date', 'lat', 'lon', 'news', 'cassualties', 'url',
       'neighborhood', 'point', 'clave_mun', 'nom_mun', 'clave_loc',
       'nom_loc','ageb']]

In [105]:
merged["clave_mun"] = merged["clave_mun"].fillna('')
merged["clave_mun"] = merged["clave_mun"].astype(str)
merged["clave_mun"] = merged["clave_mun"].str.split('.')
merged["clave_mun"] = merged["clave_mun"].str[0]

In [107]:
merged["clave_loc"] = merged["clave_loc"].fillna('')
merged["clave_loc"] = merged["clave_loc"].astype(str)
merged["clave_loc"] = merged["clave_loc"].str.split('.')
merged["clave_loc"] = merged["clave_loc"].str[0]

In [108]:
merged.to_csv("data/crime_inegi.csv", index=False)